In [110]:
import requests
import json
import time

In [87]:
# fetching games list from Free2game
def get_free2game_games():
    url = ("https://www.freetogame.com/api/games")

    response = requests.get(url)

    if not response.status_code == 200:
        print("ERROR, request not OK")
    
    data = response.json()
    return data

In [88]:
data = get_free2game_games()

In [27]:
# storing the api data in local json file
with open("free2game_games.json", 'w') as my_file:
    data_string = json.dumps(data)
    print(data_string, file=my_file)

In [ ]:
# fetching games list from steam api

In [82]:
def get_steam_games():
    url = ("https://api.steampowered.com/ISteamApps/GetAppList/v2/")

    response = requests.get(url)

    if not response.status_code == 200:
        print("ERROR, request not OK")
    
    data = response.json()
    return data

In [83]:
steam_data = get_steam_games()

In [84]:
ids = [item.get("appid") for item in steam_data['applist']['apps']]

In [111]:
len(ids)

164509

In [102]:
first_batch = ids[0:82255]

In [99]:
first_batch_update = ids[1741:10000]

In [103]:
def get_steam_games_info(gameId):
    url = ("https://store.steampowered.com/api/appdetails?appids="+f"{gameId}")

    response = requests.get(url)

    if not response.status_code == 200:
        print("ERROR, request not OK",gameId)
    else:
        data = response.json()
        return data

In [ ]:
with open("steam_games_first_batch.json", 'w') as f:
    for appid in first_batch:
        game_info = get_steam_games_info(appid)
        data_string= json.dumps(game_info)
        print(data_string,file=f)
        time.sleep(3)

In [ ]:
####### steam insights ########

In [118]:
with open("steam_games_first_batch.json", 'r') as steam:
    steam_json = steam.read()

In [122]:
data = [json.loads(line)
        for line in open('steam_games_first_batch.json', 'r', encoding='utf-8')]


In [184]:
combined_dict = {}
for item in data:
    combined_dict.update(item)
    


In [205]:
# getting all the games which has information : success = true
success_data = {}
for key, value in combined_dict.items():
    success = value.get("success")
    if success:
        success_data[key] = value

In [256]:
# total games for which api information is available
len(success_data)

17504

In [258]:
## getting types of games in steam
game_types_counts = {}

for app_id, game_info in success_data.items():
    game_type = game_info['data'].get('type', 'Unknown')
    if game_type in game_types_counts:
        game_types_counts[game_type] += 1
    else:
        game_types_counts[game_type] = 1

print("Game Types and Their Counts:")
for game_type, count in game_types_counts.items():
    print(game_type, ":", count)

Game Types and Their Counts:
game : 10825
dlc : 3852
demo : 2048
music : 771
mod : 6
episode : 2


In [230]:
# get only mod / game / dlc types of games
solid_games = {}

for app_id, game_info in success_data.items():
    game_type = game_info['data']['type']
    if (game_type == 'mod'or game_type == 'game' or game_type == 'dlc'):
        solid_games[app_id] = game_info



In [233]:
len(solid_games)

14683

In [242]:
# describing all the genres available in steam
genre_descriptions = {}

for app_id, game_info in success_data.items():
    genres = game_info['data'].get('genres', [])
    for genre in genres:
        genre_id = genre.get('id')
        genre_description = genre.get('description')
        if genre_id and genre_description:
            genre_descriptions[genre_id] = genre_description

print("Genre Descriptions:")
for genre_id, description in genre_descriptions.items():
    print(f"Genre ID: {genre_id}, Description: {description}")


Genre Descriptions:
Genre ID: 1, Description: Action
Genre ID: 4, Description: Casual
Genre ID: 23, Description: Indie
Genre ID: 70, Description: Early Access
Genre ID: 28, Description: Simulation
Genre ID: 2, Description: Strategy
Genre ID: 25, Description: Adventure
Genre ID: 37, Description: Free to Play
Genre ID: 29, Description: Massively Multiplayer
Genre ID: 3, Description: RPG
Genre ID: 18, Description: Sports
Genre ID: 9, Description: Racing
Genre ID: 58, Description: Video Production
Genre ID: 53, Description: Design & Illustration
Genre ID: 59, Description: Web Publishing
Genre ID: 51, Description: Animation & Modeling
Genre ID: 54, Description: Education
Genre ID: 57, Description: Utilities
Genre ID: 60, Description: Game Development
Genre ID: 56, Description: Software Training
Genre ID: 55, Description: Photo Editing
Genre ID: 52, Description: Audio Production


In [241]:
# getting counts of each genre
genre_counts = {}

for app_id, game_info in success_data.items():
    genres = game_info['data'].get('genres', [])
    for genre in genres:
        genre_id = genre.get('id')
        genre_description = genre.get('description')
        if genre_id and genre_description:
            if genre_id in genre_counts:
                genre_counts[genre_id] += 1
            else:
                genre_counts[genre_id] = 1

print("Genre Counts:")
for genre_id, count in genre_counts.items():
    print(f"Genre ID: {genre_id}, Count: {count}")

Genre Counts:
Genre ID: 1, Count: 5496
Genre ID: 4, Count: 5505
Genre ID: 23, Count: 8694
Genre ID: 70, Count: 1201
Genre ID: 28, Count: 3113
Genre ID: 2, Count: 2923
Genre ID: 25, Count: 5323
Genre ID: 37, Count: 1600
Genre ID: 29, Count: 459
Genre ID: 3, Count: 3221
Genre ID: 18, Count: 535
Genre ID: 9, Count: 515
Genre ID: 58, Count: 61
Genre ID: 53, Count: 367
Genre ID: 59, Count: 318
Genre ID: 51, Count: 93
Genre ID: 54, Count: 131
Genre ID: 57, Count: 121
Genre ID: 60, Count: 143
Genre ID: 56, Count: 72
Genre ID: 55, Count: 20
Genre ID: 52, Count: 22


In [261]:
# Dictionary to store description and count
description_dict = {}  

for app_id, description in genre_descriptions.items():
    if app_id in genre_counts:
        count = genre_counts[app_id]
        description_dict[description] = count

description_dict

{'Action': 5496,
 'Casual': 5505,
 'Indie': 8694,
 'Early Access': 1201,
 'Simulation': 3113,
 'Strategy': 2923,
 'Adventure': 5323,
 'Free to Play': 1600,
 'Massively Multiplayer': 459,
 'RPG': 3221,
 'Sports': 535,
 'Racing': 515,
 'Video Production': 61,
 'Design & Illustration': 367,
 'Web Publishing': 318,
 'Animation & Modeling': 93,
 'Education': 131,
 'Utilities': 121,
 'Game Development': 143,
 'Software Training': 72,
 'Photo Editing': 20,
 'Audio Production': 22}

In [253]:
from collections import defaultdict
import operator

developer_counts = defaultdict(int)

for app_id, game_info in success_data.items():
    developers = game_info['data'].get('developers', [])
    for developer in developers:
        developer_counts[developer] += 1

top_10_developers = sorted(developer_counts.items(), key=operator.itemgetter(1), reverse=True)[:10]

print("Top 10 Developers:")
for developer, count in top_10_developers:
    print(f"Developer: {developer}, Count: {count}")


Top 10 Developers:
Developer: SmiteWorks USA, LLC, Count: 289
Developer: Codename Entertainment Inc., Count: 108
Developer: CAPCOM Co., Ltd., Count: 106
Developer: Nihon Falcom, Count: 89
Developer: Gotcha Gotcha Games, Count: 64
Developer: Atomic Fabrik, Count: 59
Developer: Beenox, Count: 54
Developer: High Moon Studios, Count: 54
Developer: Raven Software, Count: 53
Developer: Treyarch, Count: 51


In [254]:
# getting the count of free games available on steam
free_games_count = 0

for app_id, game_info in success_data.items():
    is_free = game_info['data'].get('is_free', False)
    if is_free is True:
        free_games_count += 1

print("Count of Free Games:", free_games_count)

Count of Free Games: 3875
